In [1]:
import json, requests, threading
import getpass
from bs4 import BeautifulSoup
from Requester import Requester

### Get Password

In [ ]:
password = getpass.getpass(prompt="Enter Password Plis: ")

### Create Session

In [ ]:
session = requests.Session()
response = session.post("https://ayamel.byu.edu/auth/password", {"username":"admin", "password":password}, allow_redirects=False)
print(session.cookies.items())
response = session.get("https://ayamel.byu.edu/admin/content")
session.close()

### Extract Data from Content (html)

In [ ]:
class Content:
    def __init__(self, resourceId, name, owner, lang, content_type):
        self.resourceId = resourceId
        self.name = name
        self.owner = owner
        self.lang = lang
        self.content_type = content_type

def extract(row):
    fields = row.find_all('td')
    rid = fields[6].text
    name = fields[1].text
    owner = fields[2].text
    lang = fields[3].text
    content_type = fields[7].text
    return Content(rid, name, owner, lang, content_type)

In [ ]:
soup = BeautifulSoup(response.text, "html5lib")
rows = soup.find_all('tr')[1:] # get rid of the first row cause some idiots didn't use table headers
content_list = [extract(x) for x in rows]

### Get Resources

In [ ]:
from Requester import CONTENT_OBJS
from Requester import RESOURCE_OBJS

In [ ]:
NUM_THREADS = 20

In [ ]:
def run_request_process():
    pool = []
    for i in range(NUM_THREADS): pool.append(Requester())
    for t in pool: t.start()
    for t in pool: t.join()

In [ ]:
CONTENT_OBJS.extend(content_list) #wedontcare

In [ ]:
run_request_process()

In [ ]:
RESOURCE_OBJS